# NGC Catalogのコンテナを実行する--PyTorch-Singularity

---

構築したOpenHPC環境でNGC Catalogの[PyTorchコンテナ](https://ngc.nvidia.com/catalog/containers/nvidia:pytorch)を実行します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境のマスターノードに対してSSHでログインできる
* Singularityがロードされる設定になっている
* SlurmのGeneric Resource(GRES)としてGPUが登録されていること
* GPUがCUDA Compute Capability 5.2以上であること

> [AWS P2](https://aws.amazon.com/jp/ec2/instance-types/p2/)などのNVIDIA K80は[Compute Capability](https://developer.nvidia.com/cuda-gpus)が3.7なので、このNotebookで利用しているコンテナイメージを実行することができません。

マスターノードに対して SSH でログインできることを確認します。マスターノードのIPアドレスを指定してください。

In [ ]:
# (例)
# master_address = '172.30.XXX.xxx'

master_address = 

SSHでログインするユーザ名を指定してください。

In [ ]:
# (例)
# user = 'vcp'
# user = 'user00'

user = 

必要であればSSHの秘密鍵を指定してください。

In [ ]:
# (例)
# ssh_identity = '~/.ssh/id_rsa'

SSHでログインする際の引数を、変数に格納しておきます。

In [ ]:
# ユーザ名とホスト名
target = f'{user}@{master_address}'
print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ssh_identity}' if 'ssh_identity' in vars() else ''
print(ssh_opts)

マスターノードに対してSSHでログインしてコマンドを実行してみます。

In [ ]:
!ssh {ssh_opts} {target} hostname

Singularityがロードされていることを確認します。

In [ ]:
!ssh {ssh_opts} {target} module is-loaded singularity

上のセルがエラーになった場合は Singularity をロードする設定が行われていません。「061-Singularityのロード.ipynb」を実行して Singularity がロードされるよう設定してください。

SlurmのGeneric Resource(GRES)としてGPUが登録されていることを確認します。

In [ ]:
!ssh {ssh_opts} {target} sinfo -N --Format=NodeHost,Gres | grep -w gpu

上のセルがエラーになった場合はSlurmにGPUがGRESとして登録されていません。「032-設定ファイルの編集-GRESの登録.ipynb」を実行してGRESの設定をしてください。

## PyTorchコンテナの実行

PyTorchコンテナでMNISTを実行してみます。

### コンテナイメージの取得

OpenHPC環境でコンテナを実行するにはSingularityを利用します。はじめにNGCカタログから[PyTorch](https://ngc.nvidia.com/catalog/containers/nvidia:pytorch)のコンテナイメージを取得します。

> 作業領域として `/tmp` に 30GB程度の空き領域が必要となります。他のディレクトリを作業領域として利用する場合は環境変数`SINGULARITY_TMPDIR`を指定してください。詳細についてはSingularityのドキュメント [Build Customization](https://singularity.lbl.gov/build-environment#environment-variables) を参照してください。
> 
> またイメージの取得、変換には３０分以上かかります。

In [ ]:
ngc_version = '23.10'
sif_file = f'pytorch_{ngc_version}-py3.sif'

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'test -f {sif_file} || singularity pull docker://nvcr.io/nvidia/pytorch:{ngc_version}-py3'"

### 準備

MNISTのデータやスクリプトを準備します。

まず、データやスクリプトを配置するディレクトリを作成します。

In [ ]:
work_dir = 'pytorch'
!ssh {ssh_opts} {target} mkdir -p {work_dir}

データをダウンロードするためのスクリプトを配置します。

In [ ]:
!scp {ssh_opts} template/pytorch/download_mnist.py {target}:{work_dir}

スクリプトを実行し、データをダウンロードします。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && singularity exec ~/{sif_file} python download_mnist.py'"

データが取得できたことを確認します。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && ls -lR ../data'"

GitHubの [pytorch/examples](https://github.com/pytorch/examples) からMNISTの[スクリプト](https://github.com/pytorch/examples/blob/master/mnist/main.py)を取得します。

In [ ]:
mnist_url = 'https://raw.githubusercontent.com/pytorch/examples/master/mnist/main.py'
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && curl -L -o mnist_classify.py {mnist_url}'"

取得したスクリプトの内容を先頭部分を表示してみます。

In [ ]:
mnist_url = 'https://raw.githubusercontent.com/pytorch/examples/master/mnist/main.py'
!ssh {ssh_opts} {target} head {work_dir}/mnist_classify.py

### コンテナからGPUが利用できることを確認する

Singularityで実行したコンテナ環境からGPUを利用できることを確認します。

GPUを利用できるかをチェックするスクリプトを配置します。

In [ ]:
!scp {ssh_opts} template/pytorch/check_gpu.py {target}:{work_dir}

GPU利用の可否をチェックするスクリプトを実行します。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && srun -l -N 1 \
    singularity exec --nv ~/{sif_file} python check_gpu.py'"

### ジョブの実行

MNISTのスクリプトをSlurmのジョブとして実行します。

ジョブの実行スクリプトを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as workdir:
    batch_file = Path(workdir) / 'pytorch_mnist.job'
    with batch_file.open(mode='w') as f:
        f.write(f'''#!/bin/bash
        
#SBATCH -J pytorch-mnist         # create a short name for your job
#SBATCH -o pytorch-mnist.%j.out  # Name of stdout output file (%j expands to jobId)
#SBATCH -N 1                     # Total number of nodes requested
#SBATCH -n 1                     # Total number of across all nodes
#SBATCH --gres=gpu:1             # number of gpus per node
#SBATCH -t 00:10:00              # Run time (hh:mm:ss)

cd $HOME/{work_dir}
singularity exec --nv $HOME/{sif_file} python3 mnist_classify.py --epochs=3
''')
    !cat {batch_file}
    !scp {ssh_opts} {str(batch_file)} {target}:{work_dir}

ジョブを実行する前のキューの状態を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

ノードのGPU利用状況を確認します。

In [ ]:
!ssh {ssh_opts} {target} sinfo --Node --Format=NodeHost,Gres,GresUsed

ジョブを実行します。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && sbatch pytorch_mnist.job'"

ジョブの実行状況を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

ノードのGPU利用状況を確認します。`GRES_USED` の欄でノードのGPU利用状況を確認してください。

In [ ]:
!ssh {ssh_opts} {target} sinfo --Node --Format=NodeHost,Gres,GresUsed

ジョブが完了するまで数分かかります。ジョブ実行中のCPU, メモリ, GPUなどの利用状況は VCC の Grafana で確認することができます。

ジョブの完了後に次のセルを実行してください。ジョブの出力結果が確認できます。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && tail pytorch-mnist*.out'"

ジョブの実行が完了していれば出力結果の末尾に以下のような内容が表示されます。

```
Train Epoch: 3 [58240/60000 (97%)]	Loss: 0.057342
Train Epoch: 3 [58880/60000 (98%)]	Loss: 0.065162
Train Epoch: 3 [59520/60000 (99%)]	Loss: 0.010134

Test set: Average loss: 0.0321, Accuracy: 9906/10000 (99%)
```